In [1]:
import os
from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import write
import math

# Function to load audio and noise
def load_audio(file_path):
    audio = AudioSegment.from_wav(file_path)
    return audio

# Function to resample and match duration of the noise to the input audio
def match_audio_and_noise(audio, noise):
    # Match sample rate
    if audio.frame_rate != noise.frame_rate:
        noise = noise.set_frame_rate(audio.frame_rate)
    
    # Match channels: if audio is stereo (2 channels), make sure noise is also stereo (2 channels)
    if audio.channels != noise.channels:
        if audio.channels == 2:
            noise = noise.set_channels(2)  # Duplicate mono noise to stereo
        else:
            noise = noise.set_channels(1)  # Convert stereo noise to mono
    
    # Repeat or trim the noise to match the audio duration exactly
    noise = noise[:len(audio)]  # Ensure noise is the exact length of the input audio
    
    return noise

# Function to calculate RMS (Root Mean Square) of an audio signal
def rms(audio):
    samples = np.array(audio.get_array_of_samples())
    return np.sqrt(np.mean(samples**2))

# Function to calculate the noise amplitude to achieve a desired SNR (dB)
def calculate_noise_amplitude(audio, noise, snr_db):
    signal_rms = rms(audio)
    noise_rms = rms(noise)
    
    # Calculate desired noise RMS to achieve the given SNR
    desired_noise_rms = signal_rms / (10**(snr_db / 20.0))
    
    # Calculate the scaling factor
    scaling_factor = desired_noise_rms / noise_rms
    return scaling_factor

# Function to add noise to audio at a controlled SNR
def add_noise_to_audio(audio, noise, snr_db):
    # Match the noise and audio properties (duration, sample rate, and channels)
    noise = match_audio_and_noise(audio, noise)
    
    # Calculate the scaling factor for noise to achieve the desired SNR
    scaling_factor = calculate_noise_amplitude(audio, noise, snr_db)
    
    # Convert audio and noise to numpy arrays
    audio_np = np.array(audio.get_array_of_samples())
    noise_np = np.array(noise.get_array_of_samples())
    
    # Pad the shorter array to match the length of the longer one
    if len(audio_np) < len(noise_np):
        noise_np = noise_np[:len(audio_np)]  # Trim noise to match audio length
    elif len(noise_np) < len(audio_np):
        noise_np = np.pad(noise_np, (0, len(audio_np) - len(noise_np)), 'constant')  # Pad noise to match audio length
    
    # Scale the noise by the calculated scaling factor
    noise_np = noise_np * scaling_factor
    
    # Add noise to the audio (AUGMENTATION) - no change in duration here
    noisy_audio_np = audio_np + noise_np
    
    # Clip to ensure values are within the allowable range
    noisy_audio_np = np.clip(noisy_audio_np, -32768, 32767).astype(np.int16)
    
    return noisy_audio_np

# Function to save the noisy audio to file while preserving original properties
def save_noisy_audio(file_path, noisy_audio, frame_rate, channels, sample_width):
    # Re-create AudioSegment from numpy array to preserve channels and sample width correctly
    noisy_audio_segment = AudioSegment(
        noisy_audio.tobytes(),
        frame_rate=frame_rate,
        sample_width=sample_width,
        channels=channels
    )
    
    # Export the noisy audio to a WAV file
    noisy_audio_segment.export(file_path, format="wav")

# Main function
def augment_audio_with_noise(input_file, noise_file, output_file, snr_db=10):
    # Load input audio and noise
    audio = load_audio(input_file)
    noise = load_audio(noise_file)
    
    # Add noise to the audio at the given SNR (dB)
    noisy_audio = add_noise_to_audio(audio, noise, snr_db)
    
    # Ensure that the noisy audio matches the input audio duration (in samples, after resampling)
    audio_np = np.array(audio.get_array_of_samples())
    noisy_audio_np = noisy_audio[:len(audio_np)]  # Trim to match length of input audio
    
    # Save the noisy audio while maintaining the original properties
    save_noisy_audio(output_file, noisy_audio_np, audio.frame_rate, audio.channels, audio.sample_width)
    
    # Get the full path of the output file
    output_folder_path = os.path.dirname(os.path.abspath(output_file))
    
    # If the output file path is relative, make it absolute based on current working directory
    if not os.path.isabs(output_file):
        output_folder_path = os.path.join(os.getcwd(), output_folder_path)
    
    # Print the success message with file path and names
    print(f"Noise-augmented file {output_file} saved in {output_folder_path} (with original file {os.path.basename(input_file)})\n\n\n\n\n\n\n\n")

# Example usage
input_file = r'C:\Program Files (x86)\sox-14.4.2-win32\sox-14.4.2\input.wav'
noise_file = r'C:\Program Files (x86)\sox-14.4.2-win32\sox-14.4.2\destroyerengine.wav'
output_file = 'input_noisy.wav'

augment_audio_with_noise(input_file, noise_file, output_file, snr_db=15)  # Change SNR as needed


Noise-augmented file input_noisy.wav saved in C:\Users\sukal (with original file input.wav)










In [31]:
import os
from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import write
import math
# AudioSegment.ffmpeg = r"C:\Users\sukal\radioconda\Library\bin\ffmpeg.exe"

# Function to load audio and noise
def load_audio(file_path):
    audio = AudioSegment.from_file(file_path)
    return audio

# Function to resample and match duration of the noise to the input audio
def match_audio_and_noise(audio, noise):
    # Match sample rate
    if audio.frame_rate != noise.frame_rate:
        noise = noise.set_frame_rate(audio.frame_rate)
    
    # Match channels: if audio is stereo (2 channels), make sure noise is also stereo (2 channels)
    if audio.channels != noise.channels:
        if audio.channels == 2:
            noise = noise.set_channels(2)  # Duplicate mono noise to stereo
        else:
            noise = noise.set_channels(1)  # Convert stereo noise to mono
    
    # Repeat or trim the noise to match the audio duration exactly
    noise = noise[:len(audio)]  # Ensure noise is the exact length of the input audio
    
    return noise

# Function to calculate RMS (Root Mean Square) of an audio signal
def rms(audio):
    samples = np.array(audio.get_array_of_samples())
    return np.sqrt(np.mean(samples**2))

# Function to calculate the noise amplitude to achieve a desired SNR (dB)
def calculate_noise_amplitude(audio, noise, snr_db):
    signal_rms = rms(audio)
    noise_rms = rms(noise)
    
    # Calculate desired noise RMS to achieve the given SNR
    desired_noise_rms = signal_rms / (10**(snr_db / 20.0))
    
    # Calculate the scaling factor
    scaling_factor = desired_noise_rms / noise_rms
    return scaling_factor

# Function to add noise to audio at a controlled SNR
def add_noise_to_audio(audio, noise, snr_db):
    # Match the noise and audio properties (duration, sample rate, and channels)
    noise = match_audio_and_noise(audio, noise)
    
    # Calculate the scaling factor for noise to achieve the desired SNR
    scaling_factor = calculate_noise_amplitude(audio, noise, snr_db)
    
    # Convert audio and noise to numpy arrays
    audio_np = np.array(audio.get_array_of_samples())
    noise_np = np.array(noise.get_array_of_samples())
    
    # Pad the shorter array to match the length of the longer one
    if len(audio_np) < len(noise_np):
        noise_np = noise_np[:len(audio_np)]  # Trim noise to match audio length
    elif len(noise_np) < len(audio_np):
        noise_np = np.pad(noise_np, (0, len(audio_np) - len(noise_np)), 'constant')  # Pad noise to match audio length
    
    # Scale the noise by the calculated scaling factor
    noise_np = noise_np * scaling_factor
    
    # Add noise to the audio (AUGMENTATION) - no change in duration here
    noisy_audio_np = audio_np + noise_np
    
    # Clip to ensure values are within the allowable range
    noisy_audio_np = np.clip(noisy_audio_np, -32768, 32767).astype(np.int16)
    
    return noisy_audio_np

# Function to save the noisy audio to file while preserving original properties
def save_noisy_audio(file_path, noisy_audio, frame_rate, channels, sample_width):
    # Re-create AudioSegment from numpy array to preserve channels and sample width correctly
    noisy_audio_segment = AudioSegment(
        noisy_audio.tobytes(),
        frame_rate=frame_rate,
        sample_width=sample_width,
        channels=channels
    )
    
    # Export the noisy audio to a WAV file
    noisy_audio_segment.export(file_path, format="wav")

# Main function to process a single file
def augment_audio_with_noise(input_file, noise_file, output_file, snr_db=10):
    # Load input audio and noise
    audio = load_audio(input_file)
    noise = load_audio(noise_file)
    
    # Add noise to the audio at the given SNR (dB)
    noisy_audio = add_noise_to_audio(audio, noise, snr_db)
    
    # Ensure that the noisy audio matches the input audio duration (in samples, after resampling)
    audio_np = np.array(audio.get_array_of_samples())
    noisy_audio_np = noisy_audio[:len(audio_np)]  # Trim to match length of input audio
    
    # Save the noisy audio while maintaining the original properties
    save_noisy_audio(output_file, noisy_audio_np, audio.frame_rate, audio.channels, audio.sample_width)
    
    # Get the full path of the output file
    output_folder_path = os.path.dirname(os.path.abspath(output_file))
    
    # If the output file path is relative, make it absolute based on current working directory
    if not os.path.isabs(output_file):
        output_folder_path = os.path.join(os.getcwd(), output_folder_path)
    
    # Print the success message with file path and names
    print(f"Noise-augmented file {output_file} saved in {output_folder_path} (with original file {os.path.basename(input_file)})")

def process_folder(input_folder, output_folder, noise_file, snr_db=10):
    # Ensure the input folder exists
    if not os.path.exists(input_folder):
        print(f"Error: The input folder does not exist: {input_folder}")
        return
    
    # Ensure the output folder exists, and if not, create it
    if not os.path.exists(output_folder):
        print(f"Warning: Output folder does not exist. Creating: {output_folder}")
        os.makedirs(output_folder)

    # Print all files in the input folder for debugging
    print(f"Files in input folder: {os.listdir(input_folder)}")

    # Loop through all WAV files in the input folder
    for filename in os.listdir(input_folder):
        print(f"Processing file: {filename}")  # Debugging print
        
        if filename.endswith(".wav"):  # Process only WAV files
            input_file = os.path.join(input_folder, filename)  # Get the full input file path
            print(f"Found WAV file: {input_file}")  # Debugging print
            
            # Construct the output filename with "_noisy" suffix and full file path
            output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_noisy.wav")
            print(f"Going to save as WAV file: {output_file}")  # Debugging print
            
            # Process the file
            augment_audio_with_noise(input_file, noise_file, output_file, snr_db)

# Folder usage
input_folder = r"C:\Users\sukal\Downloads\Speech Sample\Control\Control"  # Path to input folder containing .wav files
output_folder = r"C:\Users\sukal\Downloads\Noise Augmented Speech Sample (Babble Noise)\Control_Noise"  # Path to output folder to save noisy audio
noise_file = r"C:\Users\sukal\Downloads\Noise Sample\Noise Sample\babble.wav"  # Path to the noise file
snr_db = 15  # Desired SNR in dB

process_folder(input_folder, output_folder, noise_file, snr_db)

print(f"\n\n\n\n\n\n\n\n")


Files in input folder: ['002-0.wav', '002-1.wav', '002-2.wav', '002-3.wav', '006-2.wav', '006-3.wav', '006-4.wav', '013-0.wav', '013-2.wav', '013-3.wav', '013-4.wav', '015-0.wav', '015-1.wav', '015-2.wav', '015-3.wav', '015-4.wav', '017-4.wav', '021-0.wav', '021-1.wav', '021-2.wav', '021-3.wav', '021-4.wav', '022-0.wav', '022-1.wav', '022-2.wav', '028-1.wav', '028-4.wav', '034-0.wav', '034-1.wav', '034-2.wav', '034-3.wav', '034-4.wav', '042-1.wav', '042-2.wav', '042-3.wav', '042-4.wav', '045-0.wav', '045-2.wav', '045-3.wav', '052-0.wav', '052-2.wav', '054-0.wav', '055-0.wav', '056-0.wav', '056-3.wav', '056-4.wav', '059-2.wav', '059-3.wav', '059-4.wav', '068-0.wav', '068-2.wav', '068-3.wav', '071-0.wav', '071-1.wav', '071-2.wav', '071-3.wav', '071-4.wav', '073-0.wav', '073-1.wav', '073-3.wav', '086-0.wav', '086-1.wav', '086-2.wav', '086-3.wav', '086-4.wav', '092-0.wav', '092-1.wav', '092-2.wav', '092-3.wav', '093-0.wav', '093-1.wav', '096-1.wav', '096-2.wav', '105-0.wav', '105-1.wav', '

In [ ]:





IGNORE FROM HERE. THESE ARE JUST SYSTEM CODE CHECKS MEANT FOR COMPATIBILITY.







In [11]:
import sys
print(sys.executable)


C:\Users\sukal\anaconda3\python.exe


In [75]:
from pydub import AudioSegment

# Test loading an audio file
audio = AudioSegment.from_file(r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\001-0.wav", format="wav", )

print(audio)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [13]:
import pydub
print(pydub.__file__)


C:\Users\sukal\anaconda3\Lib\site-packages\pydub\__init__.py


In [15]:
from pydub import AudioSegment

# Try to load an audio file (make sure you have a valid file)
audio = AudioSegment.from_file(r"C:\Users\sukal\input_noisy.wav")

# Check if the code runs successfully
print("Audio loaded successfully")


Audio loaded successfully


In [79]:
import os
from pydub import AudioSegment
from pydub.utils import which

# Manually set the ffmpeg path
# AudioSegment.ffmpeg = r"C:\Users\sukal\radioconda\Library\bin\ffmpeg.exe"  # Use the correct path to ffmpeg

# Ensure ffmpeg is found by pydub
ffmpeg_path = which("ffmpeg")
if ffmpeg_path is None:
    print("ffmpeg not found! Please install ffmpeg.")
else:
    print(f"Using ffmpeg located at: {ffmpeg_path}")

# Input folder and file
input_folder = r'C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia'
filename = '493-0.wav'

# Full file path
input_file_path = os.path.join(input_folder, filename)

# Try loading the audio file into an AudioSegment object
try:
    input_file = AudioSegment.from_wav(input_file_path)
    print("Audio file loaded successfully")
except Exception as e:
    print(f"Error loading audio file: {e}")


ffmpeg not found! Please install ffmpeg.
Error loading audio file: [WinError 2] The system cannot find the file specified


In [77]:
from pydub.utils import get_audio_segment_class

print(get_audio_segment_class().ffmpeg)  # This will show the path pydub is using for ffmpeg


ImportError: cannot import name 'get_audio_segment_class' from 'pydub.utils' (C:\Users\sukal\anaconda3\Lib\site-packages\pydub\utils.py)

In [23]:
from pydub import AudioSegment

# Try loading an audio file (make sure you have a valid file path)
input_file_path = r'C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav'
print(input_file_path)
try:
    audio = AudioSegment.from_wav(input_file_path)
    print("Audio file loaded successfully")
except Exception as e:
    print("Error loading audio:", e)


C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav
Error loading audio: [WinError 2] The system cannot find the file specified


In [25]:
import os
from pydub import AudioSegment

def process_folder(input_folder, output_folder, noise_file, snr_db):
    # Check if input folder exists
    if not os.path.exists(input_folder):
        print(f"Input folder does not exist: {input_folder}")
        return
    
    # Check if output folder exists, create if not
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all WAV files in input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".wav"):  # Process only WAV files
            input_file_path = os.path.join(input_folder, filename)  # Get full input file path
            print(f"Attempting to load file from: {input_file_path}")  # Debug print
            
            # Check if the file exists
            if os.path.exists(input_file_path):
                try:
                    # Load audio file
                    input_file = AudioSegment.from_file(input_file_path)
                    print(f"Successfully loaded: {filename}")
                    
                    # Construct output filename with "_noisy" suffix
                    output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_noisy.wav")
                    
                    # Process the file (this is where your actual noise augmentation logic goes)
                    augment_audio_with_noise(input_file, noise_file, output_file, snr_db)
                    print(f"Processed and saved noisy file: {output_file}")
                
                except Exception as e:
                    print(f"Error loading audio file {filename}: {e}")
            
            else:
                print(f"File not found: {input_file_path}")

# Dummy function for augmenting audio (replace with your actual logic)
def augment_audio_with_noise(input_audio, noise_file, output_file, snr_db):
    print("This function will augment the audio with noise. Implement the logic here.")
    input_audio.export(output_file, format="wav")
    print(f"Exported augmented audio to {output_file}")

# Example usage
input_folder = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia"  # Path to the input folder with WAV files
output_folder = r"C:\Users\sukal\Downloads\Noise Augmented Speech Sample (White Noise)\Dementia_Noise"  # Path to the folder where output files will be saved
noise_file = r"C:\Users\sukal\Downloads\Noise Sample\Noise Sample\white.wav"  # Path to the noise file
snr_db = 10  # Desired SNR in dB

# Call the function to process the folder
process_folder(input_folder, output_folder, noise_file, snr_db)


Attempting to load file from: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\001-0.wav
Error loading audio file 001-0.wav: [WinError 2] The system cannot find the file specified
Attempting to load file from: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\001-2.wav
Error loading audio file 001-2.wav: [WinError 2] The system cannot find the file specified
Attempting to load file from: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\003-0.wav
Error loading audio file 003-0.wav: [WinError 2] The system cannot find the file specified
Attempting to load file from: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\005-0.wav
Error loading audio file 005-0.wav: [WinError 2] The system cannot find the file specified
Attempting to load file from: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\005-2.wav
Error loading audio file 005-2.wav: [WinError 2] The system cannot find the file specified
Attempting to load file from: C:\Users\sukal\Downloads\Speec

In [27]:
import os
input_folder = r'C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia'
print("Files in input folder:", os.listdir(input_folder))


Files in input folder: ['001-0.wav', '001-2.wav', '003-0.wav', '005-0.wav', '005-2.wav', '007-1.wav', '007-3.wav', '010-0.wav', '010-1.wav', '010-2.wav', '010-3.wav', '010-4.wav', '014-2.wav', '016-0.wav', '016-1.wav', '016-3.wav', '016-4.wav', '018-0.wav', '023-0.wav', '023-2.wav', '024-1.wav', '024-2.wav', '029-0.wav', '029-1.wav', '030-0.wav', '030-1.wav', '033-0.wav', '033-1.wav', '033-2.wav', '033-3.wav', '033-4.wav', '035-0.wav', '035-1.wav', '039-0.wav', '043-0.wav', '046-0.wav', '046-2.wav', '049-0.wav', '049-1.wav', '050-0.wav', '051-0.wav', '051-1.wav', '051-2.wav', '051-3.wav', '053-1.wav', '057-0.wav', '057-1.wav', '057-2.wav', '058-0.wav', '058-1.wav', '058-3.wav', '058-4.wav', '061-0.wav', '061-1.wav', '062-0.wav', '065-0.wav', '065-1.wav', '065-2.wav', '066-0.wav', '067-1.wav', '067-2.wav', '070-2.wav', '076-0.wav', '076-2.wav', '076-4.wav', '078-0.wav', '078-1.wav', '087-1.wav', '089-0.wav', '091-0.wav', '091-1.wav', '091-2.wav', '094-1.wav', '094-2.wav', '094-3.wav', '

In [29]:
from pydub import AudioSegment

# Replace with the full path of a known file
file_path = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav"
print(f"Attempting to load: {file_path}")

try:
    audio = AudioSegment.from_file(file_path)
    print("Audio loaded successfully.")
except Exception as e:
    print(f"Error: {e}")


Attempting to load: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav
Error: [WinError 2] The system cannot find the file specified


In [31]:
from pydub import AudioSegment
print(AudioSegment.ffmpeg)


C:\Users\sukal\radioconda\Library\bin\ffmpeg.exe


In [33]:
from pydub.utils import mediainfo

file_path = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav"

# Manually set the ffmpeg path
AudioSegment.ffmpeg = r"C:\Users\sukal\anaconda3\Lib\site-packages\pydub\ffmpeg.exe"  # Use the correct path to ffmpeg

try:
    # This checks the audio file info using ffmpeg directly
    info = mediainfo(file_path)
    print("Audio File Info:", info)
except Exception as e:
    print(f"Error loading audio file: {e}")


Error loading audio file: [WinError 2] The system cannot find the file specified


In [35]:
import os

file_path = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav"

if os.path.exists(file_path):
    print("File found!")
else:
    print("File not found!")


File found!


In [37]:
from pydub import AudioSegment

file_path = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav"

try:
    audio = AudioSegment.from_file(file_path, format="wav")
    print("File loaded successfully!")
except Exception as e:
    print(f"Error loading file: {e}")


Error loading file: [WinError 2] The system cannot find the file specified


In [39]:
import ffmpeg

file_path = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav"

try:
    # Use ffmpeg to probe the audio file and get detailed info
    probe = ffmpeg.probe(file_path)
    print("Audio File Info:", probe)
except ffmpeg.Error as e:
    print(f"Error loading audio file: {e}")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [43]:
from pydub import AudioSegment
from pydub.utils import mediainfo

# Ensure ffmpeg is correctly set in pydub
AudioSegment.ffmpeg = r"C:\Users\sukal\anaconda3\Lib\site-packages\pydub\ffmpeg.exe"  # Replace with the actual path if needed

file_path = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia\362-0.wav"

try:
    # Retrieve and display audio file info
    info = mediainfo(file_path)
    print("Audio File Info:", info)
except Exception as e:
    print(f"Error loading audio file: {e}")


Error loading audio file: [WinError 2] The system cannot find the file specified


In [61]:
print(f"Input file: {input_file}")
print(f"Noise file: {noise_file}")


Input file: C:\Program Files (x86)\sox-14.4.2-win32\sox-14.4.2\input.wav
Noise file: C:\Users\sukal\Downloads\Noise Sample\Noise Sample\white.wav


In [11]:
# Example code in Jupyter Notebook

import os
input_folder = r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia"  # Path to the input folder
filename = "001-0.wav"  # Filename from os.listdir

input_file = os.path.join(input_folder, filename)  # Full path

# Define load_audio function
from pydub import AudioSegment

def load_audio(file_path):
    audio = AudioSegment.from_file(file_path)
    return audio

# Load the audio
audio = load_audio(input_file)

# Check the result (audio information)
print(audio)
display(audio)
